<a href="https://colab.research.google.com/github/felp99/Codecamp-Project/blob/main/IBOV_vs_SELIC_Qual_melhor_investimento%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
res_selic = requests.get('https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados?formato=json')
res_ibov = yf.Ticker('^BVSP')

In [ ]:
selic_df = pd.DataFrame(res_selic.json())
selic_df['data'] = pd.to_datetime(selic_df['data'], format='%Y/%m/%d', errors='ignore')
selic_df['SELIC'] = selic_df['valor'].astype(float)
selic_df.drop(columns=['valor'], inplace = True)
selic_df.set_index('data', inplace = True)
selic_df.head(10)

,SELIC
data,
04/06/1986,0.065041
05/06/1986,0.067397
06/06/1986,0.066740
09/06/1986,0.068247
10/06/1986,0.067041
11/06/1986,0.066740
12/06/1986,0.066575
13/06/1986,0.066164
16/06/1986,0.066493


In [ ]:
ibov_df = res_ibov.history(start="1986-01-01", end=None)
ibov_df.drop(columns=['Open', 'High', 'Low',	'Volume',	'Dividends',	'Stock Splits'], inplace=True)
ibov_df.rename(columns={'Close':'IBOV'}, inplace=True)
ibov_df.index = ibov_df.index.strftime('%d/%m/%Y')

ibov_df

,IBOV
Date,
27/04/1993,24.500000
28/04/1993,24.299999
29/04/1993,23.700001
30/04/1993,24.100000
03/05/1993,24.100000
...,...
15/01/2024,131521.000000
16/01/2024,129294.000000
17/01/2024,128524.000000


## Definindo data de início e fim

In [ ]:
df = pd.merge(ibov_df, selic_df, left_index=True, right_index=True)
init = '15/01/2020'
final = None
df = df[init : ]
df.head()

,IBOV,SELIC
15/01/2020,116414.0,0.017089
16/01/2020,116704.0,0.017089
17/01/2020,118478.0,0.017089
20/01/2020,118862.0,0.017089
21/01/2020,117026.0,0.017089


## Lógica de crescimento

In [ ]:
# Para o IBOV

df['IBOV'] = df['IBOV'].pct_change() + 1
df['SELIC'] = (df['SELIC']/100) + 1

In [ ]:
df_cumprod = df.cumprod()

In [ ]:
INITIAL_INVEST = 10000
fig = px.line(df_cumprod * INITIAL_INVEST, title='SELIC vs IBOV')
fig.show()

In [ ]:
fig = px.imshow(df_cumprod.corr())
fig.show()